# Preparation

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, FileReadTool

topics = [
    "Elon Musk's xAI Unveils Grok-3 AI Chatbot - 02/18/2025",
    "Inside OpenAI's Headquarters: Advancements Towards Artificial General Intelligence - 02/16/2025"
]

with open('covered_topics.txt', 'w') as file:
    for topic in topics:
        file.write(f"{topic}\n")

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
topics_read_tool = FileReadTool(file_path='./covered_topics.txt')

In [5]:
from datetime import date

current_date = date.today()

# Agents

In [6]:
agent_researcher = Agent(
    role="Hot Tech and AI News Topics Researcher",
    goal="Finds and ranks trending topics in AI and Technology based on the latest news.",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(f""" 
        You are a skilled researcher specializing in AI and tech news. Your primary job is to find 
        the most relevant and up-to-date topics that are trending in AI and technology. You must ensure 
        that the news you uncover is current, specifically focusing on groundbreaking stories and trends 
        that have emerged **today** ({current_date}) or in the past **few days**. Your focus is on 
        uncovering content that will captivate readers and reflect the latest advancements or discussions 
        in AI and tech.
    """),
    allow_delegation=False

)

In [7]:
agent_topic_selector = Agent(
    role="Topic Selection Strategist",
    goal="Selects the most relevant, up-to-date, and engaging AI topic while avoiding repetition of previously covered topics.",
    tools=[topics_read_tool],
    verbose=True,
    backstory=(f"""
        You are a strategic expert specializing in identifying the most impactful and trending AI topics of the day. 
        Your primary focus is to ensure that the topics you choose are **up-to-date** and aligned with the latest developments in AI, especially those that have emerged today ({current_date}). 
        You evaluate a list of current, relevant topics and cross-reference them with previously covered subjects to ensure fresh and compelling content. 
        Your goal is to avoid repetition and maintain the content's relevance, ensuring it captures the audience's attention with the most timely and interesting discussions.
    """),
    allow_delegation=False
)

In [8]:
agent_writer = Agent(
    role="LinkedIn Content Creator",
    goal="Crafts engaging and professional LinkedIn posts based on trending tech topics.",
    verbose=True,
    tools=[search_tool, scrape_tool],
    backstory=(f""" 
        You are an expert content creator with a deep understanding of LinkedIn's audience. Your role is 
        to write concise, engaging, and insightful posts on the latest tech topics. You create content that 
        is informative yet captivating, using a young, tech-savvy tone, occasional emojis, and a positive vibe. 
        Your posts should be professional, concise, and resonant with tech professionals and enthusiasts. 
        Occasionally, you can share personal insights or opinions, but always with a balanced and professional approach.
        Today is {current_date}.
    """),
    allow_delegation=False
)

In [9]:
agent_fact_checker = Agent(
    role="AI Fact Checker",
    goal="Ensure the accuracy, credibility, and factual consistency of AI-generated LinkedIn posts before publication.",
    verbose=True,
    tools=[search_tool, scrape_tool],
    backstory=(""" 
        You are a meticulous fact-checker specializing in AI and technology. Your job is to ensure that 
        LinkedIn posts are not only factually accurate but also credible and well-researched. You verify 
        claims by cross-checking with reliable sources, correcting any inaccuracies or misleading statements. 
        You also ensure the post maintains logical consistency, is coherent, and aligns with professional standards. 
        Your focus is to ensure the content is trustworthy and ready for publication while preserving the engaging tone.
        
    """),
    allow_delegation=False
)

In [10]:
agent_editor = Agent(
    role="LinkedIn Post Editor",
    goal="Refines and polishes LinkedIn posts to ensure clarity, engagement, and professional tone.",
    verbose=True,
    backstory=(""" 
        As an expert editor, your job is to ensure that each LinkedIn post is not only well-structured and clear but also 
        highly engaging and professionally written. You enhance the post's readability, flow, and overall impact, ensuring 
        it resonates with the target audience. You are responsible for the final output, making sure that the tone is 
        appropriate, concise, and captivating, while maintaining the post's overall message and purpose.
    """),
    allow_delegation=False
)


# Tasks

In [11]:
task_research = Task(
    description=(f"""
        Search the web for the most relevant and trending topics in AI, technology, and AI politics for today, 
        which is {current_date}. Always provide a **ranked list of 5 topics**. If one of these topics is particularly 
        **important** or widely discussed in the AI space today, mark it as "**Very Important**" in the list. 
        If no topic stands out as exceptionally crucial, just ensure the 5 topics are all **trending** and **interesting**.
        Please ensure that all topics are **up-to-date**, relevant to today's discussions, and **accurately reflect current trends**.
    """),
    expected_output="A ranked list of 5 trending topics, with one marked as 'Very Important' if applicable.",
    agent=agent_researcher
)

In [12]:
task_select_topic = Task(
    description=(""" 
        Analyze the ranked topics from the research and compare them with those in 'covered_topics.txt'. 
        Prioritize selecting the **very important** topic from the list, provided it has not been covered recently. 
        If the very important topic has already been covered, then select the most important topic from the remaining list that 
        has not been covered recently. Ensure you choose only one unique and specific topic for the LinkedIn post.
        The agent will use the 'covered_topics.txt' file to check recently covered topics.
    """),
    expected_output="One unique, relevant topic selected from the research that has not been covered recently, with priority given to the 'Very Important' topic.",
    agent=agent_topic_selector,
    context=[task_research]
)

In [13]:
task_write_post = Task(
    description=(f"""
        Write a concise, LinkedIn-style post about the selected topic, ensuring it is informative, 
        engaging, and professional. The post should be tailored to the audience on LinkedIn—focused on providing value 
        and insights, not a blog-style article. If the topic is a **hot topic** of today ({current_date}), make sure to 
        emphasize its relevance. If the topic has been trending for several days or weeks, adapt the post accordingly 
        while keeping it fresh and timely. Feel free to use emojis where appropriate to enhance engagement, but maintain a 
        professional tone throughout. 
    """),
    expected_output="A well-written LinkedIn post of 150-200 words, clear and engaging.",
    agent=agent_writer,
    context=[task_select_topic],
)

In [14]:
task_fact_check_post = Task(
    description=(f"""
        Review the LinkedIn post about the selected topic for factual accuracy, credibility, 
        and logical consistency. Cross-check all claims, statistics, and references with reliable, up-to-date sources. 
        Provide feedback only if you explicitly find inaccuracies, outdated information, or misleading statements. 
        If no issues are found, no feedback should be provided. Ensure that the post maintains its engaging and professional tone 
        while remaining factually accurate. Recommendations should only be given if specific factual issues are identified, 
        and they should be actionable and straightforward.
    """),
    expected_output="A LinkedIn post with identified factual issues (if any) corrected, and recommendations where applicable.",
    agent=agent_fact_checker,
    context=[task_write_post],
)

In [15]:
task_edit_post = Task(
    description=(""" 
        Review the LinkedIn post for clarity, grammar, tone, and overall engagement. 
        Take into account any feedback from the fact-checking process, making sure to remove or correct any 
        information flagged as inaccurate, outdated, or misleading. 
        Ensure the post remains professional, on-brand, and well-structured, improving readability, flow, and conciseness. 
        The final version should be polished, clear, and ready for publication.
        After editing, save the final version of the post to the 'post.md' file.
        **Do not add "post saved as 'post.md'" at the end of the document**
    """),
    expected_output="A well-polished LinkedIn post, free from grammatical errors, with corrected factual information, ready for publication",
    agent=agent_editor,
    context=[task_write_post, task_fact_check_post],
    output_file="post.md",
)


# Crew

In [16]:
community_management_crew = Crew(
    agents=[agent_researcher, agent_topic_selector, agent_writer, agent_fact_checker, agent_editor],
    tasks=[task_research, task_select_topic, task_write_post, task_fact_check_post, task_edit_post],
    verbose=True
)

# Run

In [17]:
result = community_management_crew.kickoff()

 [DEBUG]: == Working Agent: Hot Tech and AI News Topics Researcher
 [INFO]: == Starting Task: 
        Search the web for the most relevant and trending topics in AI, technology, and AI politics for today, 
        which is 2025-02-18. Always provide a **ranked list of 5 topics**. If one of these topics is particularly 
        **important** or widely discussed in the AI space today, mark it as "**Very Important**" in the list. 
        If no topic stands out as exceptionally crucial, just ensure the 5 topics are all **trending** and **interesting**.
        Please ensure that all topics are **up-to-date**, relevant to today's discussions, and **accurately reflect current trends**.
    


> Entering new CrewAgentExecutor chain...
I need to start by searching the internet for the most relevant and trending topics in AI, technology, and AI politics for today, 2025-02-18. I should focus on finding groundbreaking stories and trends that have emerged in the past few days.

Action: Search th

I need to write a compelling LinkedIn post about the news regarding Mira Murati's AI startup and its recruitment of top researchers from competitors.

Action: Read website content
Action Input: {"website_url": "https://www.technews.com/mira-murati-ai-startup-recruits-top-researchers"} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='www.technews.com', port=443): Max retries exceeded with url: /mira-murati-ai-startup-recruits-top-researchers (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'www.technews.com'. (_ssl.c:1006)"))).
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: Let's try searching the internet for more information on Mira Murati's AI startup and its recruitment of top researchers from competitors.
Action: Sear

The original post is engaging and informative, but it can be improved by enhancing clarity and conciseness. I will refine the content to ensure it resonates with the target audience and maintains a professional tone.

Final Answer:
🚀 Exciting Tech News Alert 🚀 Former OpenAI technology chief Mira Murati's AI startup, Thinking Machines Lab, is attracting top researchers from competitors with about 30 brilliant minds joining the team. Founded by Mira Murati, the website of Thinking Machines Lab confirms her as the founder and highlights the startup's commitment to innovation and excellence in the AI space. It's inspiring to see former OpenAI employees like Murati and Barret Zoph coming together to drive the future of AI forward. Stay tuned for more updates on this cutting-edge venture that promises to push the boundaries of artificial intelligence and shape the tech landscape of tomorrow. 🌐💥

#AI #TechInnovation #FutureTech #MiraMurati #ThinkingMachinesLab

> Finished chain.
 [DEBUG]: == 

In [18]:
from IPython.display import Markdown, display
display(Markdown("./post.md"))

🚀 Exciting Tech News Alert 🚀 Former OpenAI technology chief Mira Murati's AI startup, Thinking Machines Lab, is attracting top researchers from competitors with about 30 brilliant minds joining the team. Founded by Mira Murati, the website of Thinking Machines Lab confirms her as the founder and highlights the startup's commitment to innovation and excellence in the AI space. It's inspiring to see former OpenAI employees like Murati and Barret Zoph coming together to drive the future of AI forward. Stay tuned for more updates on this cutting-edge venture that promises to push the boundaries of artificial intelligence and shape the tech landscape of tomorrow. 🌐💥

#AI #TechInnovation #FutureTech #MiraMurati #ThinkingMachinesLab